In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from config import protocol, username, password, host, port

## Extract CSV's inot Dataframes

In [ ]:
zips = "Resources/uszips.csv"
tax = "Resources/tax_info.csv"

In [ ]:
zipcode_df = pd.read_csv(zips)
zipcode_df.head()

In [ ]:
tax_df = pd.read_csv(tax)
tax_df.head()

## Transform Data

In [ ]:
zipcode_cols = ["zip", "state_name", "city", "population", "density"]
zipcode_transformed = zipcode_df[zipcode_cols].copy()

#rename columns headers
zipcode_transformed = zipcode_transformed.rename(columns ={'zip': "zip_code",
                                                            "state_name": "state",
                                                            "city": "city",
                                                            "population":"population",
                                                            "density":"population_density"})

zipcode_transformed.drop_duplicates
# zipcode_transformed.set_index('zip', inplace=True)

zipcode_transformed.head()

In [ ]:
tax_df = tax_df[["STATE", "zipcode", "A00100", "agi_stub"]]
#A00100
tax_df

In [ ]:
tax_cols = ["STATE", "zipcode", "A00100", "agi_stub"]
tax_transformed = tax_df[tax_cols].copy()

#rename columns headers
tax_transformed = tax_transformed.rename(columns ={"STATE": "state",
                                                            "zipcode": "zip_code",
                                                            "A00100":"adjusted_gross_income",
                                                               "agi_stub": 'tax_bracket'
                                                            })

tax_transformed.drop_duplicates

tax_transformed.head()

In [ ]:
tax_transformed = tax_transformed.loc[tax_transformed['zip_code'] != 0]
tax_transformed.reset_index(inplace=True, drop=True)
tax_transformed

## Database Conection

In [ ]:
database_name = 'tax_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
engine.table_names()

In [ ]:
zipcode_transformed.to_sql(name='us_zips', con=engine, if_exists='append', index=False)

In [ ]:
tax_transformed.to_sql(name='tax_info', con=engine, if_exists='append', index=False)